In [ ]:
import pandas as pd
import numpy as np
import regex
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean, cosine
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
dolly_cqa_df = pd.read_parquet('./data/dolly_cqa_prepared.parquet')
dolly_cqa_df

In [ ]:
domain_pattern = r'\baseball(?!: bat)|basketball|badminton|tennis(?!: shoe)|soccer|futbol|football (?!: stadium)'
domain_flags = [bool(regex.search(domain_pattern, sent, regex.IGNORECASE)) for sent in dolly_cqa_df['text']]

task_pattern = r'what (is|are)'
task_flags = [bool(regex.search(task_pattern, sent, regex.IGNORECASE)) for sent in dolly_cqa_df['text']]

structure_pattern = r'\b(given a|given the|the given|based on|reference text|as a reference|the following text)'
structure_flags = [bool(regex.search(structure_pattern, sent, regex.IGNORECASE)) for sent in dolly_cqa_df['text']]

dolly_cqa_df['domain_flag'] = domain_flags
dolly_cqa_df['task_flag'] = task_flags
dolly_cqa_df['structure_flag'] = structure_flags

In [ ]:
def train_lr_classifier(df, embeddings_col, y_col):
    # Convert the list of lists in X_colname to a numpy array
    X = np.array(df[embeddings_col].tolist())
    
    y = df[y_col]
    
    # Initialize LR classifier
    clf = LogisticRegressionCV(cv=5, scoring='f1_macro', max_iter=10000, n_jobs=-1)

    # Fit the classifier
    clf.fit(X, y)

    return clf

In [ ]:
clf_domain = train_lr_classifier(df=dolly_cqa_df, embeddings_col='embeddings_mini_lm', y_col='domain_flag')
clf_task = train_lr_classifier(df=dolly_cqa_df, embeddings_col='embeddings_mini_lm', y_col='task_flag')
clf_structure = train_lr_classifier(df=dolly_cqa_df, embeddings_col='embeddings_mini_lm', y_col='structure_flag')

In [ ]:
def get_predictions(df, clf, embeddings_col, y_col):
    # Convert the list of lists in X_colname to a numpy array
    X = np.array(df[embeddings_col].tolist())

    # Normalize X
    X = X / np.linalg.norm(X, axis=1, keepdims=True)
    
    y = df[y_col]

    # Predictions
    y_pred = clf.predict(X)

    # Printing the classification report
    print(classification_report(y, y_pred))

    # Plotting the confusion matrix
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(7,5))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
get_predictions(df=dolly_cqa_df, clf=clf_domain, embeddings_col='embeddings_mini_lm', y_col='domain_flag')

In [ ]:
get_predictions(df=dolly_cqa_df, clf=clf_task, embeddings_col='embeddings_mini_lm', y_col='task_flag')

In [ ]:
get_predictions(df=dolly_cqa_df, clf=clf_structure, embeddings_col='embeddings_mini_lm', y_col='structure_flag')

In [ ]:
n_clusters = 10

In [ ]:
model = clf_task

dolly_cqa_df['score'] = model.predict_proba(X=np.array(dolly_cqa_df['embeddings_mini_lm'].tolist()))[:,1]
dolly_cqa_df['w_embeddings'] = dolly_cqa_df['embeddings_mini_lm'].apply(lambda x: x * model.coef_[0])
dolly_cqa_df['iw_embeddings'] = dolly_cqa_df['embeddings_mini_lm'].apply(lambda x: x / model.coef_[0])

seed=0
uw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['embeddings_mini_lm']])
w_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['w_embeddings']])
iw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df['iw_embeddings']])

dolly_cqa_df['uw_kmeans'] = uw_kmeans.labels_
dolly_cqa_df['w_kmeans'] = w_kmeans.labels_
dolly_cqa_df['iw_kmeans'] = iw_kmeans.labels_

cluster_df = dolly_cqa_df[['uw_kmeans', 'w_kmeans', 'iw_kmeans', 'score']]
cluster_df = pd.melt(cluster_df, id_vars=['score'], var_name='weighting', value_name='cluster_id')
cluster_df = cluster_df.groupby(["weighting", "cluster_id"]).agg({'score': ['mean', 'var']}).reset_index()
cluster_df.columns = ['weighting', 'cluster_id', 'mean_score', 'var_score']

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='mean_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='mean_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Mean Score by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='var_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='var_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Variance of Scores by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
def plot_cluster_scores(df):
  
    cluster_columns = ['uw_kmeans', 'w_kmeans', 'iw_kmeans']
    
    # Set up the matplotlib figure
    num_clusters = len(cluster_columns)
    fig, axs = plt.subplots(num_clusters, 1, figsize=(8, 4*num_clusters))
    
    for i, cluster_column in enumerate(cluster_columns):
        ax = axs[i]
        sns.countplot(data=df, x=cluster_column, hue='task_flag', ax=ax)
        ax.set_title(f'Score Counts by {cluster_column}')
        ax.set_xlabel('Cluster')
        ax.set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_scores(dolly_cqa_df)

In [ ]:
def _compute_cohesion(cluster_embeddings, metric_cosine):
    n = len(cluster_embeddings)
    if n < 2:
        return 0  # Cannot compute cohesion for a cluster with less than two points
    if metric_cosine:
        total_distance = sum(cosine(cluster_embeddings[i] / np.linalg.norm(cluster_embeddings[i]),
                                    cluster_embeddings[j] / np.linalg.norm(cluster_embeddings[j])) for i in range(n) for j in range(i+1, n))
    else:
        total_distance = sum(euclidean(cluster_embeddings[i] / np.linalg.norm(cluster_embeddings[i]),
                                       cluster_embeddings[j] / np.linalg.norm(cluster_embeddings[j])) for i in range(n) for j in range(i+1, n))
    num_pairs = n * (n - 1) // 2  # Number of unique pairs in the cluster
    return total_distance / num_pairs  # Normalize by the number of pairs

def plot_cluster_cohesion(df, metric_cosine):
    
    cluster_columns = ['uw_kmeans', 'w_kmeans', 'iw_kmeans']
    embedding_columns = ['embeddings_mini_lm', 'w_embeddings', 'iw_embeddings']

    num_cluster_types = len(cluster_columns)
    fig, axs = plt.subplots(num_cluster_types, 1, figsize=(6, 3*num_cluster_types))

    for i, (cluster_column, embedding_column) in enumerate(zip(cluster_columns, embedding_columns)):
        df['cohesion_'+cluster_column] = 0
        for cluster_id in df[cluster_column].unique():
            data = df[embedding_column][df[cluster_column] == cluster_id].reset_index(drop=True)
            cohesion = _compute_cohesion(data, metric_cosine)
            df['cohesion_'+cluster_column][df[cluster_column] == cluster_id] = cohesion
        
        ax = axs[i]
        sns.barplot(data=df.groupby(cluster_column)['cohesion_'+cluster_column].mean().reset_index(), x=cluster_column, y='cohesion_'+cluster_column, ax=ax)
        ax.set_title(f'Normalized Cohesion by {cluster_column}')
        ax.set_xlabel('Cluster')
        ax.set_ylabel('Cohesion')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_cohesion(dolly_cqa_df, metric_cosine=False)

In [ ]:
def _compute_separation(cluster_embeddings_1, cluster_embeddings_2, metric_cosine):
    if metric_cosine:
        total_distance = sum(cosine(point1 / np.linalg.norm(point1),
                                    point2 / np.linalg.norm(point2)) for point1 in cluster_embeddings_1 for point2 in cluster_embeddings_2)
    else:
        total_distance = sum(euclidean(point1 / np.linalg.norm(point1),
                                       point2 / np.linalg.norm(point2)) for point1 in cluster_embeddings_1 for point2 in cluster_embeddings_2)
    num_pairs = len(cluster_embeddings_1) * len(cluster_embeddings_2)  # Number of pairs between the clusters
    return total_distance / num_pairs  # Normalize by the number of pairs

def plot_cluster_separation(df, metric_cosine):
    
    cluster_columns = ['uw_kmeans', 'w_kmeans', 'iw_kmeans']
    embedding_columns = ['embeddings_mini_lm', 'w_embeddings', 'iw_embeddings']

    num_cluster_types = len(cluster_columns)
    fig, axs = plt.subplots(num_cluster_types, 1, figsize=(8, 4*num_cluster_types))

    for n, (cluster_column, embedding_column) in enumerate(zip(cluster_columns, embedding_columns)):
        num_clusters = len(df[cluster_column].unique())
        separation_matrix = np.zeros((num_clusters, num_clusters))
        
        for i in range(num_clusters):
            for j in range(i+1, num_clusters):
                data_i = df[embedding_column][df[cluster_column] == i].reset_index(drop=True)
                data_j = df[embedding_column][df[cluster_column] == j].reset_index(drop=True)
                separation = _compute_separation(data_i, data_j, metric_cosine)
                separation_matrix[i, j] = separation
                separation_matrix[j, i] = separation  # The matrix is symmetric
        
        ax = axs[n]
        sns.heatmap(separation_matrix, annot=True, cmap='viridis', fmt=".2f", linewidths=.5, ax=ax)
        ax.set_title('Inter-Cluster Normalized Separation Heatmap for Cluster ' + cluster_column)
        ax.set_xlabel('Cluster Index')
        ax.set_ylabel('Cluster Index')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_separation(dolly_cqa_df, metric_cosine=False)

In [ ]:
def plot_3d(df, categories, embeddings, reduction='T-SNE'):
    df = df.copy()

    # Perform dimensionality reduction
    if reduction == 'PCA':
        reducer = PCA(n_components=3)
    elif reduction == 'T-SNE':
        reducer = TSNE(n_components=3)
    else:
        raise ValueError('Invalid dimensionality reduction method. Use "PCA" or "T-SNE".')

    # Extract embeddings and apply reduction
    embeddings = np.stack(df[embeddings].values)
    reduced_embeddings = reducer.fit_transform(embeddings)

    # Add reduced embeddings to dataframe
    df['x'] = reduced_embeddings[:, 0]
    df['y'] = reduced_embeddings[:, 1]
    df['z'] = reduced_embeddings[:, 2]
    df['categ'] = df[categories].astype(str)

    # Create the 3D plot using plotly.express
    fig = px.scatter_3d(df, x='x', y='y', z='z',
                        color='categ', opacity=0.5,
                        size=[1] * len(df),
                        width=800, height=600)

    fig.show()

In [ ]:
plot_3d(df=dolly_cqa_df, categories='uw_kmeans', embeddings='embeddings_mini_lm', reduction='PCA' )

In [ ]:
plot_3d(df=dolly_cqa_df, categories='w_kmeans', embeddings='w_embeddings', reduction='PCA' )

In [ ]:
plot_3d(df=dolly_cqa_df, categories='iw_kmeans', embeddings='iw_embeddings', reduction='PCA' )

In [ ]:
for c in dolly_cqa_df['text'][dolly_cqa_df['iw_kmeans'] == 0]:
    print(c, '\n=============================================================')

Here is another way I think could potentially give us better results in emphasizing or de-emphasizing an aspect of interest:

1. Identify good sentences representative of the aspect of interest (e.g. a specific subject, or a particular type of question): We have this already.

2. Instead of training a linear classifier or regressor, we create an embedding vector representative of that aspect: We could simply take the mean of all embedded sentences we identify in 1. We also create an embedding vector non-representative of that aspect in the same way using the sentences not flagged in 1.

3. Re-weight the sentences embeddings: Here we need a way to modify the original sentence embeddings by adding or subtracting the influence of the desired aspect. We can do that in this way:

    3.1. Project each sentence embedding vector onto the aspect embedding vectors computed earlier, creating "positive" and "negative" projections.

    3.2 Add each projection vector to the corresponding sentence embedding (if we want to increase the aspect influence in the embeddings) or subtract each projection vector from the corresponding sentence embedding (if we want to decrease the aspect influence in the embeddings).

    3.3 When creating weighted embedding vector, we do this in a way that we try to increase the influence of the "positive" projections and decrease the influence of "negative" projections into the sentences that we consider having the aspect of interest, and we also do the opposite for the sentences not having the aspect of interest.
    
    3.4 When creating inverse-weighted embedding vectors, we do the opposite we did for the weighted vectors.

    3.5 Alpha is a hyperparameter controlling the influence of the projection vectors into the sentence embeddings. It could be optimized by searching it for the minimum entropy of the aspect by cluster when clustering with weighted vectors and at the same time searching for the maximum entropy of the aspect by cluster when clustering with inverse-weighted vectors.

In [ ]:
dolly_cqa_df_2 = dolly_cqa_df.copy()

In [ ]:
pattern_flag = 'task_flag'

dolly_cqa_df_2['score'] = dolly_cqa_df_2[pattern_flag]

pos_aspect_embedding = np.mean(dolly_cqa_df_2[dolly_cqa_df_2[pattern_flag] == True]['embeddings_mini_lm'])
pos_aspect_embedding = pos_aspect_embedding / np.linalg.norm(pos_aspect_embedding)

neg_aspect_embedding = np.mean(dolly_cqa_df_2[dolly_cqa_df_2[pattern_flag] == False]['embeddings_mini_lm'])
neg_aspect_embedding = neg_aspect_embedding / np.linalg.norm(neg_aspect_embedding)

dolly_cqa_df_2['w_embeddings'] = dolly_cqa_df_2['embeddings_mini_lm']
dolly_cqa_df_2['iw_embeddings'] = dolly_cqa_df_2['embeddings_mini_lm']

alpha = 1.25

for i, embedding in enumerate(dolly_cqa_df_2['embeddings_mini_lm']):
    embedding = embedding / np.linalg.norm(embedding)
    pos_projection = np.dot(embedding, pos_aspect_embedding.T) * pos_aspect_embedding
    pos_projection = pos_projection.reshape(-1,)
    neg_projection = np.dot(embedding, neg_aspect_embedding.T) * neg_aspect_embedding
    neg_projection = neg_projection.reshape(-1,)
    if dolly_cqa_df_2[pattern_flag].iloc[i] == True:
        dolly_cqa_df_2['w_embeddings'].iloc[i] = embedding + alpha * (pos_projection - neg_projection)
        dolly_cqa_df_2['iw_embeddings'].iloc[i] = embedding - alpha * (pos_projection - neg_projection)
    else:
        dolly_cqa_df_2['w_embeddings'].iloc[i] = embedding + alpha * (neg_projection - pos_projection)
        dolly_cqa_df_2['iw_embeddings'].iloc[i] = embedding - alpha * (neg_projection - pos_projection)

seed=0
uw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df_2['embeddings_mini_lm']])
w_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df_2['w_embeddings']])
iw_kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto").fit( [v for v in dolly_cqa_df_2['iw_embeddings']])

dolly_cqa_df_2['uw_kmeans'] = uw_kmeans.labels_
dolly_cqa_df_2['w_kmeans'] = w_kmeans.labels_
dolly_cqa_df_2['iw_kmeans'] = iw_kmeans.labels_

cluster_df = dolly_cqa_df_2[['uw_kmeans', 'w_kmeans', 'iw_kmeans', 'score']]
cluster_df = pd.melt(cluster_df, id_vars=['score'], var_name='weighting', value_name='cluster_id')
cluster_df = cluster_df.groupby(["weighting", "cluster_id"]).agg({'score': ['mean', 'var']}).reset_index()
cluster_df.columns = ['weighting', 'cluster_id', 'mean_score', 'var_score']

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='mean_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='mean_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Mean Score by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
sns.set(rc={"figure.figsize":(5, 3)})
fig, axes = plt.subplots(2, 1, sharey=True)

sns.stripplot(x='var_score', y='weighting', data=cluster_df, jitter=True, hue='weighting', dodge=True, ax=axes[0])
axes[0].legend(loc='upper right', bbox_to_anchor=(1.5, 1))
axes[0].set(ylabel=None)
axes[0].set(xticklabels=[])
axes[0].set(xlabel=None)

sns.boxplot(x='var_score', y ='weighting', data=cluster_df, hue='weighting', dodge=True, ax=axes[1])
axes[1].set(ylabel=None)
axes[1].set(xlabel='Variance of Scores by Cluster')
axes[1].legend([], [], frameon=False)

plt.show()

In [ ]:
plot_cluster_scores(dolly_cqa_df_2)

In [ ]:
plot_cluster_cohesion(dolly_cqa_df_2, metric_cosine=False)

In [ ]:
plot_cluster_separation(dolly_cqa_df_2, metric_cosine=False)

In [ ]:
plot_3d(df=dolly_cqa_df_2, categories='uw_kmeans', embeddings='embeddings_mini_lm', reduction='PCA' )

In [ ]:
plot_3d(df=dolly_cqa_df_2, categories='w_kmeans', embeddings='w_embeddings', reduction='PCA' )

In [ ]:
plot_3d(df=dolly_cqa_df_2, categories='iw_kmeans', embeddings='iw_embeddings', reduction='PCA' )

In [ ]:
for c in dolly_cqa_df_2['text'][dolly_cqa_df_2['iw_kmeans'] == 17]:
    print(c, '\n=============================================================')